# tensoflow_probability的使用

1. 学习<https://zh-v2.d2l.ai/chapter_preliminaries/probability.html>中的内容。
2. **tensorflow_probability必须在tensorflow 2.8.0以上版本才能使用**。

## 第一个例子。下面课程的逻辑步骤也非常自然
   1. 先说明如何使用一个函数，并举了一个例子。同时生成一个样本。
   2. 如何生成10个样本。
   3. 生成1000个样本，然后来看是否符合概率分布。

In [2]:
import tensorflow_probability as tfp
import tensorflow as tf
import numpy as np

### 1. 先说明如何使用一个函数，并举了一个例子。同时生成一个样本。

1. 原文：“为了抽取一个样本，即掷骰子，我们只需传入一个概率向量。 输出是另一个相同长度的向量：它在索引i处的值是采样结果中i出现的次数。”\
实践的代码如下：这个代码运行有个特点，每次运行tfp.distributions.Multinomial(1, fair_probs).sample()输出的结果并不一样。这个过程好像是在模拟丢骰(tou2)子。每次投掷骰子时，得到的结果不一定相同的。**也就是按照一定的分布生成了一次样本**。\
fair_probs这个变量的命名猜测是fair probability的缩写。平均概率。

2. 原文：“将概率分配给一些离散选择的分布称为多项分布（multinomial distribution）。”这里出现了多项式分布。这里还不理解。
3. 也就是说tfp.distributions这个库是来生成各种概率分布的。

In [9]:
fair_probs = tf.ones(6) / 6
print(fair_probs)
tfp.distributions.Multinomial(1, fair_probs).sample()

tf.Tensor([0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667], shape=(6,), dtype=float32)


<tf.Tensor: shape=(6,), dtype=float32, numpy=array([1., 0., 0., 0., 0., 0.], dtype=float32)>

### 2. 如何生成10个样本。

原文：“在估计一个骰子的公平性时，我们希望从同一分布中生成多个样本。 如果用Python的for循环来完成这个任务，速度会慢得惊人。 因此我们使用深度学习框架的函数同时抽取多个样本，得到我们想要的任意形状的独立样本数组。”也就是符合特定分布的多组样本。**可以看到第一个参数应该是样本产生的次数。第二个参数时概率分布**。

In [5]:
tfp.distributions.Multinomial(10, fair_probs).sample()

<tf.Tensor: shape=(6,), dtype=float32, numpy=array([3., 1., 1., 1., 2., 2.], dtype=float32)>

### 3. 生成1000个样本，然后来看是否符合概率分布。

In [10]:
counts = tfp.distributions.Multinomial(1000, fair_probs).sample()
counts / 1000

<tf.Tensor: shape=(6,), dtype=float32, numpy=array([0.175, 0.147, 0.178, 0.159, 0.179, 0.162], dtype=float32)>

注意sample里面的参数500。tfp.distributions.Multinomial(10, fair_probs).sample(500)表示的含义是共进行500组试验，每次抽样10个样本。

In [12]:
counts = tfp.distributions.Multinomial(10, fair_probs).sample(500)
cum_counts = tf.cumsum(counts, axis=0)
estimates = cum_counts / tf.reduce_sum(cum_counts, axis=1, keepdims=True)
print(estimates)

tf.Tensor(
[[0.2        0.         0.1        0.2        0.2        0.3       ]
 [0.15       0.15       0.1        0.15       0.15       0.3       ]
 [0.1        0.16666667 0.1        0.16666667 0.1        0.36666667]
 ...
 [0.16285141 0.15883534 0.16566265 0.17008032 0.17891566 0.16365463]
 [0.16292585 0.15871744 0.16553107 0.17034069 0.17875752 0.16372746]
 [0.1628     0.1584     0.166      0.17       0.1788     0.164     ]], shape=(500, 6), dtype=float32)


原文：“请注意，离散（discrete）随机变量（如骰子的每一面）和连续（continuous）随机变量（如人的体重和身高）之间存在微妙的区别。现实生活中，**测量两个人是否具有完全相同的身高没有太大意义**。如果我们进行足够精确的测量，你会发现这个星球上没有两个人具有完全相同的身高。在这种情况下，询问某人的身高是否落入给定的区间，比如是否在1.79米和1.81米之间更有意义。在这些情况下，我们将这个看到某个数值的可能性量化为密度（density）。高度恰好为1.80米的概率为0，但密度不是0。在任何两个不同高度之间的区间，我们都有非零的概率。在本节的其余部分中，**我们将考虑离散空间中的概率**。对于连续随机变量的概率。”